# Data Integration

This ipynb file has the logic to retrieve prices of the crypto assets that the user wants to trade with.

In [1]:
#Imports
import ccxt
import os
from Historic_Crypto import HistoricalData
from pytrends.request import TrendReq
from twilio.rest import Client
from dotenv import load_dotenv

In [2]:
# Reading Twilio environment variables
load_dotenv()
twilio_account_sid = os.getenv("twilio_account_sid")
twilio_auth_token = os.getenv("twilio_auth_token")
twilio_client = Client(twilio_account_sid, twilio_auth_token)

# Preparing ccxt engine
exchange_id = 'coinbasepro'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'timeout': 30000,
    'enableRateLimit': True
})

In [3]:
# ccxt.exchanges

In [4]:
# Function is to get OHLCV data for a ticker over 1 day period. It uses ccxt library to get the data.
# product - Crypto asset to get the data for. Eg. BTC/USD
# time - Current time in epoch
def retrieve_ohlcv(product):
    return exchange.fetch_ohlcv (product, '1m')

def get_current_price(product):
    price = retrieve_ohlcv(product)[-1][4]
    return price

In [5]:
# Function is to get historical data for a ticker as far back as the start date provided at the interval of 1 day. It uses Historic_Crypto library to get the data.
# product - Crypto asset to get the data for. Eg. BTC-USD
# start_date -  Start date as a string in the format YYYY-MM-DD-HH-MM (str)
def retrieve_historical_data(product,start_date):
    df = HistoricalData(product, 86400, start_date).retrieve_data()
    return df

In [6]:
# Function to send a SMS to the number provided. This is used to notify whether user should buy or not buy the assets.
def send_notification_via_sms(number, body):
    message = twilio_client.messages.create(
        to=number, 
        from_="+18056331101",
        body=body)
    return message.sid

In [7]:
def get_trend_data(kw):
    try:
        pytrends = TrendReq(hl='en-US', tz=600)
        pytrends.build_payload([kw], cat=0, timeframe='now 1-H', geo='', gprop='')
        df = pytrends.interest_over_time()
        return df.iloc[-1][0]
    except:
        return 0

In [ ]:

# sid = send_notification_via_sms("+61406311992", "Test")
# print(sid)

In [ ]:
# int(time.time() * 1000)

# retrieve_ohlcv('BTC/USD')[-1][4]

In [ ]:
# pytrends = TrendReq(hl='en-US', tz=600)
# kw_list = ["Buy Bitcoin"]
# kw_list_2 = ["BTC USD"]
# pytrends.build_payload(kw_list, cat=0, timeframe='now 1-H', geo='', gprop='')

In [ ]:
# pytrends.interest_over_time()

In [ ]:
# pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=False)
# kw_list = ["Buy Bitcoin"]
# get_trend_data(kw_list[0])

In [ ]:
#Code to get real time price data

# Parameters are optional
# wsClient = cbpro.WebsocketClient(url="wss://ws-feed.pro.coinbase.com",
#                                 products=["BTC-USD"],
#                                 channels=["ticker"])
# Do other stuff...
# wsClient.start()
# wsClient.close()